In [24]:
import openai
from latex2sympy2 import latex2sympy
import json

#model = "gpt-3.5-turbo-0613"
model = "gpt-4-0613"

def calc_latex_math_expression(expression):
    sympy_expr = latex2sympy(expression)
    result = sympy_expr.doit()
    return str(result)

def ask_user(question):
    return input(question)

functions = [
     {
      "name": "calc_latex_math_expression",
      "description": "Calculate a Latex mathematical expression",
      "parameters": {
        "type": "object",
        "properties": {
          "expression": {
            "type": "string",
            "description": "The Latex expression to calculate"
          },
        },
        "required": ["expression"]
      }
    },
    {
      "name": "ask_user",
      "description": "Asks the user for input",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question to ask to the user"
          },
        },
        "required": ["question"]
      }
    },
  ]

available_functions = {
    "calc_latex_math_expression": calc_latex_math_expression,
    "ask_user": ask_user,
}

In [25]:
def run_conversation(content):
  messages = [
    {"role": "user", "content": content},
    ]
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  messages.append(response_message)  # extend conversation with assistant's reply

  # Step 2: check if GPT wanted to call a function
  while response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    if function_name not in available_functions:
      raise Exception("Function " + function_name + " not found")
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Step 4: send the info on the function call and function response to GPT
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    response_message = response["choices"][0]["message"]
    messages.append(response_message)  # extend conversation with assistant's reply

  for message in messages:
    print(message["role"] + ": " + message["content"])
    if message.get("function_call"):
      print(" -> calling function " + message["function_call"]["name"] + " with arguments " + json.dumps(message["function_call"]["arguments"]))
  

In [26]:
run_conversation("If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step.")

user: If I am 42 years old and my brother is 3 years younger than me, how old is my brother? Explain step by step.
assistant: Sure, let's break it down step by step:

Step 1: Start with your age, which is 42 years old.
Step 2: The problem states that your brother is 3 years younger than you.
Step 3: To find out your brother's age, subtract 3 from your age.

This is expressed in the equation below, where A represents your age and B represents your brother's:
B = A - 3
Now let's compute it.
 -> calling function calc_latex_math_expression with arguments "{\n  \"expression\": \"42 - 3\"\n}"
function: 39
assistant: After subtracting 3 from 42, we get 39. Therefore, your brother is 39 years old.


In [27]:
run_conversation("If a circle has a radius of 3 cm, what is its symbolic area? Explain step by step.")

user: If a circle has a radius of 3 cm, what is its symbolic area? Explain step by step.
assistant: The formula for calculating the area of a circle is given by:

A = πr²

Where:
- "A" is the area of the circle
- "r" is the radius of the circle
- "π" is a constant whose approximate value is 3.14159

Step 1:
Substitute the given radius (r = 3 cm) into the formula:

A = π * 3²

Step 2: 
Square the radius (3² = 9):

A = π * 9

So, the symbolic representation of the area of this circle is 9π square cm. 

Would you like to calculate the numerical value for this?


In [28]:
run_conversation("If a sphere has a radius of 3 cm, what is its volume? Explain step by step.")

user: If a sphere has a radius of 3 cm, what is its volume? Explain step by step.
assistant: The volume V of a sphere is given by the formula:

V = 4/3 * π * r³

Where:
- r is the radius of the sphere.
- π is a mathematical constant whose approximate value is 3.14159.

Step 1: Plug the radius value into the formula.
Let's substitute r = 3 cm into the formula. This will give us:

V = 4/3 * π * 3³

Let's calculate this.
 -> calling function calc_latex_math_expression with arguments "{\n  \"expression\": \"4/3 * pi * 3^3\"\n}"
function: 36*i*p
assistant: The volume of the sphere is approximately 36π cubic cm. If you want a decimal approximation, you can calculate this as well.


In [29]:
run_conversation("If my brother is 3 years younger than me, how old is my brother? Explain step by step.")

user: If my brother is 3 years younger than me, how old is my brother? Explain step by step.
assistant: Sure, there are a few necessary steps to determine the age of your brother.

Step 1: Know your age
We need to know your current age, which we will use as our starting point.

Step 2: Perform subtraction
We subtract 3 years from your current age. This subtraction is performed because your brother is 3 years younger than you are.

Now, let's get your age first. 

Could you please tell me how old you are currently?
 -> calling function ask_user with arguments "{\n  \"question\": \"Could you please tell me how old you are currently?\"\n}"
function: 42
assistant: Thank you. 

Step 3: Now to calculate the age of your brother. This is done by subtracting 3 years from your age.

So, we will do 42 (your age) - 3 = 39

So, your brother is 39 years old.
